In [15]:
'''INITIAL CURVE SETUP
======================
1) DATA
========
- Historical Swap Data (swap_data)
- Normal Volatility Data (vol_data)
- Historical Zero Coupon Curves (zeroCoupons)
- Historical Forward Curves (forwardCurves)

2) FUNCTIONS
=============
- Zero Coupon interpolator (zeroCouponInterpolator)
- Forward Interpolator (forwardRate)
- Forward Swap Rate Interpolator (forwardSwapRate)

3) PACKAGES
=============
Set1: math, numpy, pandas, itertools, matplotlib
Set2: scipy.stats, scipy.optimize, datetime, time, relativedelta
Set3: nelson_siegel_svensson
Set4: prettytable
'''

# Initialize Curve Setup
%run CurveSetup.ipynb

# Multiprocessing
import multiprocessing as mp
from random import seed
from openpyxl import load_workbook

In [16]:
'''TEST 3: HIGHAM AND MAO
=========================='''
def fNHighamMaoCurve(forwardCurve, alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear):
    timeSteps = len(forwardCurve) * timeStepsPerYear
    dt = 1/timeSteps
  
    # Set the forward Curve
    curve = np.array(forwardCurve)
    
    # Define the volatility terms
    alpha1Exponential = sigma1 * np.exp(-alpha1 * np.arange(1, len(forwardCurve)+1))
    alpha2Exponential = sigma2 * np.exp(-alpha2 * np.arange(1, len(forwardCurve)+1))
    sigma1T =  np.sqrt(1 - np.power(rho, 2))* alpha1Exponential
    sigma2T = alpha2Exponential + rho * alpha1Exponential
    
    # Terminal Curve
    terminalCurve = []
    gaussians = []
    deflateurs = []
      
    # Loop to construct curves
    for t in range(1, timeSteps+1):
        # Generate Gaussians
        rand1 = np.random.standard_normal(len(curve))
        rand2 = np.random.standard_normal(len(curve))
        
        # Calculate Drift
        mu1 = np.array(sigma1T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma1T* np.power(curve + delta, eta)/
                                   (1 + curve)))       
        mu2 = np.array(sigma2T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma2T* np.power(curve + delta, eta)/
                                   (1 + curve)))
        mu = mu1 + mu2
                       
        # Construct Curve     
        curve = (curve
                    + np.power(np.absolute(curve) + delta, eta) * dt * mu
                    + sigma1*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*rand1 
                    + sigma2*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*rand2)
        
        # At end of each year append the curve to TerminalCurve
        if (t%timeStepsPerYear) == 0 :
            terminalCurve.append(curve)
            deflateurs.append(1/(1+curve[0]))
            curve = curve[1:] # Delete 1 year from curve
            sigma1T = sigma1T[:len(sigma1T)-1]
            sigma2T = sigma2T[:len(sigma2T)-1]
            gaussians.append(np.matrix([stats.norm.cdf(rand1), stats.norm.cdf(rand2)]))
            
    return([terminalCurve, gaussians, deflateurs])

In [61]:
'''TEST 3: HIGHAM AND MAO
=========================='''
def fNHighamMaoCurve2(forwardCurve, alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear):
    timeSteps = len(forwardCurve) * timeStepsPerYear
    dt = 1/timeSteps
  
    # Set the forward Curve
    curve = np.array(forwardCurve)
    curveAntithetique = curve
    
    # Define the volatility terms
    alpha1Exponential = sigma1 * np.exp(-alpha1 * np.arange(1, len(forwardCurve)+1))
    alpha2Exponential = sigma2 * np.exp(-alpha2 * np.arange(1, len(forwardCurve)+1))
    sigma1T =  np.sqrt(1 - np.power(rho, 2))* alpha1Exponential
    sigma2T = alpha2Exponential + rho * alpha1Exponential
    
    # Terminal Curve
    terminalCurve = []
    terminalCurveAntithetique = []
    gaussians = []
    deflateurs = []
    gaussians = np.random.standard_normal((timeSteps, 2))
    gaussiansAntithetique = - gaussians

    # Loop to construct curves
    for t in range(1, timeSteps+1):
        
        # Calculate Drift
        mu1 = np.array(sigma1T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma1T* np.power(curve + delta, eta)/
                                   (1 + curve)))       
        mu2 = np.array(sigma2T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma2T* np.power(curve + delta, eta)/
                                   (1 + curve)))
        mu = mu1 + mu2
                       
        # Construct Curve     
        curve = (curve
                    + np.power(np.absolute(curve) + delta, eta) * dt * mu
                    + sigma1*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*gaussians[t-1, 0] 
                    + sigma2*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*gaussians[t-1, 1])
        
        curveAntithetique = (curveAntithetique
                    + np.power(np.absolute(curve) + delta, eta) * dt * mu
                    + sigma1*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*gaussiansAntithetique[t-1, 0] 
                    + sigma2*np.power(np.absolute(curve)+delta, eta)*sqrt(dt)*gaussiansAntithetique[t-1, 1])
        
        # At end of each year append the curve to TerminalCurve
        if (t%timeStepsPerYear) == 0 :
            terminalCurve.append(curve)
            terminalCurveAntithetique.append(curveAntithetique)
            deflateurs.append(1/(1+curve[0]))
            curve = curve[1:] # Delete 1 year from curve
            curveAntithetique = curveAntithetique[:1]
            sigma1T = sigma1T[:len(sigma1T)-1]
            sigma2T = sigma2T[:len(sigma2T)-1]
            
    return([terminalCurve,
            terminalCurveAntithetique,
            [gaussians[:,0], gaussiansAntithetique[:,0]],
            [gaussians[:,1], gaussiansAntithetique[:,1]],
            deflateurs])

In [18]:
'''TEST 4: HULL & WHITE
=========================='''
def fHullWhiteCurve(forwardCurve, alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear):
    timeSteps = len(forwardCurve) * timeStepsPerYear
    dt = 1/timeSteps
  
    # Set the forward Curve
    curve = np.array(forwardCurve)
    qCurve = np.power(curve+delta, eta)/(1 - eta)
    
    # Define the volatility terms
    alpha1Exponential = sigma1 * np.exp(-alpha1 * np.arange(1, len(forwardCurve)+1))
    alpha2Exponential = sigma2 * np.exp(-alpha2 * np.arange(1, len(forwardCurve)+1))
    sigma1T =  np.sqrt(1 - np.power(rho, 2))* alpha1Exponential
    sigma2T = alpha2Exponential + rho * alpha1Exponential
    
    # Terminal Curve
    terminalCurve = []
    gaussians = []
    deflateurs = []
      
    # Loop to construct curves
    for t in range(1, timeSteps+1):
        # Generate Gaussians
        rand1 = np.random.standard_normal(len(curve))
        rand2 = np.random.standard_normal(len(curve))
        
        # Calculate Drift
        mu1 = np.array(sigma1T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma1T* np.power(curve + delta, eta)/
                                   (1 + curve)))       
        mu2 = np.array(sigma2T * np.power(curve + delta, eta) * 
                             np.cumsum(sigma2T* np.power(curve + delta, eta)/
                                   (1 + curve)))
        mu = mu1 + mu2
        
        fCurveSum = np.cumsum(np.power(curve + delta, eta-1))
        
        # Construct Curve     
        qCurve = (qCurve
                    + dt * mu 
                    + 0.5 *eta* (np.power(sigma1T, 2) + np.power(sigma2T, 2) + fCurveSum)
                    + sigma1T*sqrt(dt)*rand1 
                    + sigma2T*sqrt(dt)*rand2)
        
        # At end of each year append the curve to TerminalCurve
        if (t%timeStepsPerYear) == 0 :
            terminalCurve.append(curve)
            deflateurs.append(1/(1+curve[0]))
            curve = curve[1:] # Delete 1 year from curve
            qCurve = qCurve[1:]
            sigma1T = sigma1T[:len(sigma1T)-1]
            sigma2T = sigma2T[:len(sigma2T)-1]
            gaussians.append(np.matrix([stats.norm.cdf(rand1), stats.norm.cdf(rand2)]))
            
    return([terminalCurve, gaussians, deflateurs])

In [19]:
'''EIOPA CURVE
==============='''
EIOPACurve = np.array(pd.read_excel('EIOPA Curve.xlsx',
              header = None,
             index_col=0))
EIOPAZCCurve = [pow(1/(1+EIOPACurve[i-1]), i) for i in range(1, len(EIOPACurve)+1)]
EIOPAForwardCurve = [(EIOPAZCCurve[i-1]/EIOPAZCCurve[i])-1 for i in range(1, len(EIOPAZCCurve))]

In [62]:
'''INPUT PARAMETERS
====================='''
forwardCurve = EIOPAForwardCurve
zeroCouponCurve = EIOPAZCCurve
delta = 0.25
sigma1 = 0.08249                 
sigma2 = 0.10682    
alpha1 =  0.07384
alpha2 = 0.2003  
rho = -1.0 
eta =  0.53195 
  
timeStepsPerYear =  1
expiries = np.arange(1,len(forwardCurve)+1)
np.random.seed(50)

'''SIMULATION
==============='''
start = time()
simulatedCurves250 = []
simulatedCurves250Antithetique = []
gaussians1 = []
gaussians2 = []
deflateurs = []

for i in range(2500):
    results = fNHighamMaoCurve2(forwardCurve,  alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear)  
    
    # Simulate
    simulatedCurves250.append(results[0])
    simulatedCurves250Antithetique.append(results[1])
    gaussians1.append(results[2])
    gaussians2.append(results[3])
    deflateurs.append(results[4])
    
end = time()
print((end- start)/60)

stochasticDrift250 = zeroCouponCurve[0]*np.cumprod(np.nanmean(deflateurs, axis = 0))

0.6837499976158142


In [ ]:
# Table
header = ['Year', 'ZC', 'Projected', 'Errors(Abs)', 'Errors(%)']
resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, len(zeroCouponCurve)+1)])
resultsTable.add_column(header[1], list(np.round(zeroCouponCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(stochasticDrift250, 6))
resultsTable.add_column(header[3], np.round(zeroCouponCurve[1:] - stochasticDrift250,  6))
resultsTable.add_column(header[4], np.round(np.abs(zeroCouponCurve[1:]/stochasticDrift250 - 1)*100, 6))
print(resultsTable)

In [ ]:
# Plot
fig, ax = plt.subplots(figsize = (40, 36), facecolor='white')

title = 'Martingale Test - EIOPA CURVE'
# Set the title and distance from the plo
ttl = ax.title
ttl.set_position([0.5, 1.05])
ax.set_title(title, color='purple', fontsize = 40)

projPlot = plt.scatter(x = np.arange(1, len(zeroCouponCurve)),
            y =stochasticDrift250, 
                       alpha = 1,
                      color='red',
                      marker = 'o',
                      s = 100)
zcPlot = plt.scatter(x = np.arange(1, len(zeroCouponCurve)),
            y = zeroCouponCurve[1:],
           color = 'green',
           marker = 'o',
                    s= 100)

plt.xlabel('Maturity', fontsize = 36, color = 'purple')
plt.ylabel('ZC Value', fontsize = 36, color = 'purple')
ax.tick_params(labelcolor='black',
              labelsize = 30)
plt.legend((projPlot, zcPlot), 
          ('Projected', 'Actual'),
          loc = 'upper right',
          prop={'size': 45})

plt.style.use('seaborn-darkgrid')
plt.savefig("Martingale Test.png")

In [68]:
book = load_workbook('Data\\Gaussiens.xlsx')
writer = pd.ExcelWriter('Data\\Gaussiens.xlsx', engine = 'openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
for i in range(len(gaussians1)):
    pd.DataFrame(gaussians1[i]).to_excel(writer, "Gaussiens1",
                                          startcol = 1, 
                                          startrow = i*2+1,
                                          index = False, 
                                          header = False)
    pd.DataFrame(gaussians2[i]).to_excel(writer, "Gaussiens2",
                                          startcol = 1, 
                                          startrow = i*2+1,
                                          index = False, 
                                          header = False)
writer.save()

In [ ]:
'''WRITE THE RESULTS TO EXCEL
=============================='''
#Open the file
book = load_workbook('Data\\Martingale Test.xlsx')
writer = pd.ExcelWriter('Data\\Martingale Test.xlsx', engine = 'openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

# Write each line into Excel (Result - Triangles)
for i in range(len(uniforms250)):
    pd.DataFrame(zeroCoupons.loc['2019-12-31']).to_excel(writer, 
                                                         "Zero Coupons",
                                                         startcol = 2,
                                                        startrow = len(zeroCoupons.loc['2019-12-31'])*(i)+2, 
                                                        index = False,
                                                        header = False)
    
    pd.DataFrame(forwardCurves.loc['2019-12-31']).to_excel(writer, "Forwards",
                                                                   startcol = 2,
                                                                   startrow = len(simulatedCurves250[i])*(i)+2, 
                                                                  index = False,
                                                                  header = False)
    
    for j in range(1, len(uniforms250[0])+1):
        pd.DataFrame(uniforms250[i][j-1][0]).transpose().to_excel(writer, "Uniforms1",
                                          startcol = j+1, 
                                          startrow = int(j + len(uniforms250[i])*(i)),
                                        index = False, 
                                         header = False)
        
        pd.DataFrame(uniforms250[i][j-1][1]).transpose().to_excel(writer, "Uniforms2",
                                          startcol = j+1, 
                                          startrow = int(j + len(uniforms250[i])*(i)),
                                        index = False, 
                                         header = False)
        
        pd.DataFrame(simulatedCurves250[i][j-1]).to_excel(writer, "Forwards",
                                          startcol = j+2, 
                                          startrow = int(j + len(simulatedCurves250[i])*(i))+1,
                                        index = False, 
                                         header = False)
        
        pd.DataFrame(np.cumprod(1/(1+simulatedCurves250[i][j-1]))).to_excel(writer, "Zero Coupons",
                                  startcol = 2+j, 
                                  startrow = int(j + len(zeroCoupons.loc['2019-12-31'])*(i))+2,
                                  index = False, 
                                  header = False)
        
        pd.DataFrame(zeroCoupons.loc['2019-12-31'][j]* np.cumprod(1/
                        (1+simulatedCurves250[i][j-1]))).to_excel(writer, "Zero Coupons Tilde",
                          startcol = 2+j, 
                          startrow = int(j + len(zeroCoupons.loc['2019-12-31'])*(i))+2,
                          index = False, 
                          header = False)

        
# Write the initial Parameters
pd.DataFrame(zeroCoupons.loc['2019-12-31']).transpose().to_excel(writer, "Parameters",
                                                                 startcol = 1,
                                                                 startrow = 1, 
                                                                 index = False,
                                                                header = False)

pd.DataFrame(forwardCurves.loc['2019-12-31']).transpose().to_excel(writer, "Parameters",
                                                                   startcol = 1,
                                                                   startrow = 2,
                                                                  index = False,
                                                                  header = False)

pd.DataFrame([delta, eta, sigma1, sigma2]).to_excel(writer, "Parameters", 
                                                                    startcol = 1,
                                                                   startrow = 5,
                                                                  index = False,
                                                                  header = False)

writer.save()

In [ ]:
'''OBTAINING DISCOUNT CURVES FROM FORWARDS
============================================'''
# Obtain the actual Discount Factors D(0, t) = cumprod(D(0, i)) for i less than t
DF = [np.append(zeroCoupons.loc['2019-12-31'][1],
                zeroCoupons.loc['2019-12-31'][1]*np.cumprod(df)) for df in deflateurs]

# Obtain the Zero Coupon Bonds
zCoupons = list(simulatedCurves250)
zCouponsTilde = list(simulatedCurves250)

# Transform Forwards to ZC
for i in range(len(simulatedCurves250)):
    for j in range(len(simulatedCurves250[0])):
        zCoupons[i][j] = np.cumprod(1/(1+simulatedCurves250[i][j]))
        
        # Obtain discount factors for each timestep
        zCouponsTilde[i][j] = zCoupons[i][j]*DF[i][j]
        
# ZCTimeToMaturity = []
# for i in range(len(simulatedCurves250)):
#     for j in range(len(simulatedCurves250[0])):
#         ZCTimeToMaturity.append([x[j] for x in simulatedCurves250[0][:(len(simulatedCurves250[0])-j)]])

In [ ]:
resMean = list(zCoupons[0])

for i in range(len(resMean)):
    resMean[i] = [0]*len(resMean[i])
    
for i in range(len(zCouponsTilde)):
    for j in range(len(zCouponsTilde[0])):
        resMean[j] = np.nansum([resMean[j], zCouponsTilde[i][j]], axis = 0)
        
# Final Data        
zCouponTildeAvg = [i/1000 for i in resMean]

In [ ]:
book = load_workbook('Data\\Test2.xlsx')
writer = pd.ExcelWriter('Data\\Test2.xlsx', engine = 'openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

for j in range(len(ZCTimeToMaturity)):
    pd.DataFrame(ZCTimeToMaturity[j]).transpose().to_excel(writer, "Distributions",
                                     startcol = 2,
                                     startrow = 2+j, 
                                     index = False,
                                    header = False)
writer.save()

In [ ]:
zeroCouponCurve = zeroCoupons.loc['2019-12-31']
stochasticDrift250 = zeroCouponCurve[1]*np.cumprod(np.nanmean(deflateurs, axis = 0))
plt.plot(stochasticDrift250)
plt.scatter(x = np.arange(2, 60),y = zeroCouponCurve[2:])


In [ ]:
'''RESULTS
==========='''
header = ['Forward Expiry','Discount Curve','Stochastic Drift']

resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, 61)])
resultsTable.add_column(header[1], list(np.round(zeroCouponCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(stochasticDrift250, 6))
print(resultsTable)

In [ ]:
timeSteptsPerYear = 200
start = time()
simulatedCurves200 = []
uniforms200 = []
deflateurs200 = []

for i in range(1000):
    results = fNHighamMaoCurve(forwardCurve,  alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear)  
    
    # Simulate
    simulatedCurves200.append(results[0])
    uniforms200.append(results[1])
    deflateurs200.append(results[2])
    

end = time()
(end- start)/60

In [ ]:
zeroCouponCurve = zeroCoupons.loc['2019-12-31']
stochasticDrift200 = zeroCouponCurve[1]*np.cumprod((1/(1+np.nanmean(simulatedCurves200, axis = 0))))
plt.plot(stochasticDrift200)
plt.scatter(x = np.arange(2, 60),y = zeroCouponCurve[2:])

In [ ]:
'''RESULTS
==========='''
header = ['Forward Expiry','Discount Curve','Stochastic Drift200']

resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, 61)])
resultsTable.add_column(header[1], list(np.round(zeroCouponCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(stochasticDrift200, 6))
print(resultsTable)

In [ ]:
timeStepsPerYear =  150
start = time()
simulatedCurves150 = []  
for i in range(10000):
    
    # Simulate
    simulatedCurves150.append(fNHighamMaoCurve(forwardCurve, sigma1, sigma2, eta, delta, timeStepsPerYear))

end = time()
(end- start)/60

In [ ]:
zeroCouponCurve = zeroCoupons.loc['2020-02-14']
stochasticDrift150 = zeroCouponCurve[1]*np.cumprod((1/(1+np.nanmean(simulatedCurves150, axis = 0))))
plt.plot(stochasticDrift150)
plt.scatter(x = np.arange(2, 60),y = zeroCouponCurve[2:])

In [ ]:
'''RESULTS
==========='''
header = ['Forward Expiry','Discount Curve','Stochastic Drift150']

resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, 61)])
resultsTable.add_column(header[1], list(np.round(zeroCouponCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(stochasticDrift150, 6))
print(resultsTable)

In [ ]:
timeSteptsPerYear = 100
start = time()
simulatedCurves100 = []
uniforms100 = []
deflateurs100 = []

for i in range(1000):
    results = fNHighamMaoCurve(forwardCurve,  alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear)  
    
    # Simulate
    simulatedCurves100.append(results[0])
    uniforms100.append(results[1])
    deflateurs100.append(results[2])
    

end = time()
(end- start)/60

In [ ]:
zeroCouponCurve = zeroCoupons.loc['2019-12-31']
stochasticDrift100 = zeroCouponCurve[1]*np.cumprod(np.nanmean(deflateurs100, axis = 0))
plt.plot( stochasticDrift100)
plt.scatter(x = np.arange(2, 60),y = zeroCouponCurve[2:])

In [ ]:
'''RESULTS
==========='''
header = ['Forward Expiry','Discount Curve','Stochastic Drift150']

resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, 61)])
resultsTable.add_column(header[1], list(np.round(zeroCouponCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(stochasticDrift100, 6))
print(resultsTable)

In [ ]:
timeStepsPerYear =  50
start = time()
simulatedCurves50 = []  
for i in range(10000):
    
    # Simulate
    simulatedCurves50.append(fNHighamMaoCurve(forwardCurve, sigma1, sigma2, eta, delta, timeStepsPerYear))

end = time()
(end- start)/60

In [ ]:
zeroCouponCurve = zeroCoupons.loc['2020-02-14']
stochasticDrift50 = zeroCouponCurve[1]*np.cumprod((1/(1+np.nanmean(simulatedCurves50, axis = 0))))
plt.plot(stochasticDrift1)
plt.scatter(x = np.arange(2, 60),y = zeroCouponCurve[2:])

In [ ]:
'''RESULTS
==========='''
header = ['Forward Expiry','Discount Curve','Stochastic Drift150']

resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, 61)])
resultsTable.add_column(header[1], list(np.round(zeroCouponCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(stochasticDrift50, 6))
print(resultsTable)

In [ ]:
timeSteptsPerYear = 1
start = time()
simulatedCurves1 = []
uniforms1 = []
deflateurs1 = []

for i in range(1000):
    results = fNHighamMaoCurve(forwardCurve,  alpha1, alpha2, sigma1, sigma2, rho, eta, delta, timeStepsPerYear)  
    
    # Simulate
    simulatedCurves1.append(results[0])
    uniforms1.append(results[1])
    deflateurs1.append(results[2])
    

end = time()
(end- start)/60

In [ ]:
zeroCouponCurve = zeroCoupons.loc['2019-12-31']
stochasticDrift1 = zeroCouponCurve[1]*np.cumprod(np.nanmean(deflateurs1, axis = 0))
plt.plot(stochasticDrift1)
plt.scatter(x = np.arange(2, 60),y = zeroCouponCurve[2:])

In [ ]:
'''RESULTS
==========='''
header = ['Forward Expiry','Discount Curve','Stochastic Drift150']

resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, 61)])
resultsTable.add_column(header[1], list(np.round(zeroCouponCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(stochasticDrift1, 6))
print(resultsTable)

In [ ]:
longCurve = np.interp(np.arange(1, 5*250 + 1)/250, 
                      np.arange(0, 11), 
                      np.append(0, np.array(forwardCurve[:10])))
timeStepsPerYear = 1
sigma1 = 0.0001
sigma2 = 0.0001
delta = 0

start = time()
simulatedCurvesDays = []  
for i in range(1000):
    
    # Simulate
    simulatedCurvesDays.append(fNHighamMaoCurve(longCurve, sigma1, sigma2, eta, delta, timeStepsPerYear))

end = time()
(end- start)/60

In [ ]:
initialCurve = np.interp(np.arange(1, 5*250 + 1)/250, 
                         np.arange(0, 11), 
                         np.append(0, np.array(zeroCouponCurve[1:11])))
simulatedCurves= initialCurve[0]*np.cumprod((1/(1+np.nanmean(simulatedCurvesDays, axis = 0))))
initialCurve

plt.plot(longCurve, lw=2, ls='--',dash_capstyle='butt')
plt.plot(np.nanmean(simulatedCurvesDays, axis = 0))
plt.style.use('bmh')
plt.xlabel('Time')
plt.ylabel('Rate')
plt.legend(['Forward Curve', 'ZC Curve'], loc=2)
plt.show()
plt.show()

In [ ]:
plt.scatter(x = np.arange(1, 5*250 + 1)/250, y = simulatedCurves,s=1,)
plt.scatter(x = np.arange(1, 5*250 + 1)/250, y = initialCurve, s = 1)
plt.style.use('bmh')
plt.xlabel('Time')
plt.ylabel('Rate')
plt.legend(['Simulated Curve', 'ZC Curve'], loc=3)
plt.show()

In [ ]:
'''RESULTS
==========='''
header = ['Forward Expiry','Discount Curve','Stochastic Drift150']

resultsTable = PrettyTable()
resultsTable.add_column(header[0], [str(i)+'Y' for i in range(2, 1251)])
resultsTable.add_column(header[1], list(np.round(initialCurve[1:], 6)))
resultsTable.add_column(header[2], np.round(simulatedCurves[:-1], 6))
print(resultsTable)

In [ ]:
# start = time()
# absorption = list(itertools.starmap(fNAbsorption, test))
# logEuler = list(itertools.starmap(fNLogEuler2, test))
# highamMao = list(itertools.starmap(fNLogEuler2, test))
# partialTrunc = list(itertools.starmap(fNPartialTrunc, test))
# end = time()

# # Timer
# print((end - start)/60)

In [ ]:
# '''TEST 5: LOG EULER
# ====================='''
# def fNLogEuler(forwardCurve, sigma1, sigma2, eta, delta, timeStepsPerYear, expiry, numPaths):
#     timeSteps = expiry * timeStepsPerYear
#     dt = 1/timeSteps
#     fKt = np.repeat(forwardCurve[expiry], numPaths)
  
#     # Define all the frozen drift terms
#     mu = [(sigma1**2 + sigma2**2) * np.sum(1/(1 + forwardCurve[:i])) for i in range(len(forwardCurve))]
    
#     for t in range(1, timeSteps+1):
#         rand1 = np.random.standard_normal(numPaths)
#         rand2 = np.random.standard_normal(numPaths)
#         fKt = fKt*np.exp((np.power(fKt+delta, eta)/fKt)*(
# #                    (dt * np.sum(mu[int(np.trunc((t-1)/timeStepsPerYear)):expiry])+
#                     ((-0.5 *(np.power(fKt+delta, eta)/fKt)*pow(sigma1,2)*dt)+ 
#                      sigma1*rand1*sqrt(dt))+
#                     ((-0.5 *(np.power(fKt+delta, eta)/fKt)*pow(sigma2,2)*dt)+ 
#                      sigma2*rand2*sqrt(dt))))
#     return(fKt)

In [ ]:
# '''TEST 1: ABSORPTION
#  ======================'''
# def fNAbsorption(forwardCurve, sigma1, sigma2, eta, delta, timeStepsPerYear, expiry, numPaths):
#     '''Objective: Project the forward at a specific expiry'''
#     timeSteps = expiry * timeStepsPerYear
#     dt = 1/timeSteps
#     fKt = np.repeat(np.interp(expiry,np.arange(1, len(forwardCurve)+1), forwardCurve),
#                    numPaths)
    
#     # Define all the frozen drift terms
#     mu = [(sigma1**2 + sigma2**2) * np.sum(1/(1 + forwardCurve[:i])) for i in range(len(forwardCurve))]

#     for t in range(1, timeSteps+1):
#         rand1 = np.random.standard_normal(numPaths)
#         rand2 = np.random.standard_normal(numPaths)
#         fKt = np.maximum((fKt 
#                             + np.power(fKt + delta, eta) * dt * np.sum(mu[int(np.trunc((t-1)/timeStepsPerYear)):expiry])
#                             + sigma1*np.power(fKt + delta, eta)*sqrt(dt)*rand1 
#                             + sigma2*np.power(fKt+delta, eta)*sqrt(dt)*rand2),0)
#     return(fKt)

In [ ]:
# '''TEST 4: PARTIAL TRUNCATION
# =============================='''
# def fNPartialTrunc(forwardCurve, sigma1, sigma2, eta, delta, timeStepsPerYear, expiry, numPaths):
#     timeSteps = expiry * timeStepsPerYear
#     dt = 1/timeSteps
#     fKt = np.repeat(forwardCurve[expiry], numPaths)
  
#     # Define all the frozen drift terms
#     mu = [(sigma1**2 + sigma2**2) * np.sum(1/(1 + forwardCurve[:i])) for i in range(len(forwardCurve))]
    
#     for t in range(1, timeSteps+1):
#         rand1 = np.random.standard_normal(numPaths)
#         rand2 = np.random.standard_normal(numPaths)
#         fKt = (fKt
#                     + np.power(np.maximum(fKt, 0) + delta, eta) * dt * 
#                            np.sum(mu[int(np.trunc((t-1)/timeStepsPerYear)):expiry])
#                             + sigma1*np.power(np.maximum(fKt,0)+delta, eta)*sqrt(dt)*rand1 
#                             + sigma2*np.power(np.maximum(fKt,0)+delta, eta)*sqrt(dt)*rand2)
#     return(fKt)

In [ ]:
# def fNLogEuler2(forwardCurve, sigma1, sigma2, eta, delta, timeStepsPerYear, expiry, numPaths):
#     timeSteps = expiry * timeStepsPerYear
#     dt = 1/timeSteps
    
#     # Define all the frozen drift terms
#     mu = [(sigma1**2 + sigma2**2) * np.sum(1/(1 + forwardCurve[:i])) for i in range(len(forwardCurve))]
    
#     paths =  np.zeros((timeSteps+1, numPaths), np.float64)
#     paths[0] = forwardCurve[expiry]
    
#     for t in range(1, timeSteps+1):
#         rand1 = np.random.standard_normal(numPaths)
#         rand2 = np.random.standard_normal(numPaths)
#         paths[t] = paths[t-1]*np.exp((np.power(paths[t-1]+delta, eta)/paths[t-1])*
#                    (dt * np.sum(mu[int(np.trunc((t-1)/timeStepsPerYear)):expiry])+
#                        ((-0.5 *(np.power(paths[t-1]+delta, eta)/paths[t-1])*pow(sigma1,2)*dt)+ 
#                      sigma1*rand1*sqrt(dt))+
#                     ((-0.5 *(np.power(paths[t-1]+delta, eta)/paths[t-1])*pow(sigma2,2)*dt)+ 
#                      sigma2*rand2*sqrt(dt))))
    
#     results  = paths[timeSteps] 
    
#     return(results)

In [ ]:
# '''TEST 3: HIGHAM AND MAO
# # =========================='''
# def fNHighamMao(forwardCurve, sigma1, sigma2, eta, delta, timeStepsPerYear, expiry, numPaths):
#     '''Objective: Project the forward at a specific expiry'''
#     timeSteps = expiry * timeStepsPerYear
#     dt = 1/timeSteps
#     fKt = np.repeat(np.interp(expiry,np.arange(1, len(forwardCurve)+1), forwardCurve),
#                    numPaths * 2)

#     # Define all the frozen drift terms
#     mu = [(sigma1**2 + sigma2**2) * np.sum(np.power(forwardCurve[:i]+ delta, eta)/
#                                        (1 + forwardCurve[:i])) for i in range(1, len(forwardCurve)+1)]  
   
#     for t in range(1, timeSteps+1):
#         rand1 = np.random.standard_normal(numPaths)
#         rand1 = np.append(rand1, -rand1)
#         rand2 = np.random.standard_normal(numPaths)
#         rand2 = np.append(rand2, -rand2)
#         fKt = (fKt
#                 + np.power(np.absolute(fKt) + delta, eta) * dt * 
# #                np.sum(mu[int(np.trunc((t-1)/timeStepsPerYear)):expiry])
#                             + sigma1*np.power(np.absolute(fKt)+delta, eta)*sqrt(dt)*rand1 
#                             + sigma2*np.power(np.absolute(fKt)+delta, eta)*sqrt(dt)*rand2)
#     return(fKt)